# Imports

In [ ]:
#suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from nltk import word_tokenize
from pattern.en import suggest
import pandas as pd
from pycontractions import Contractions
%run NER_Functions.py
import re
import nltk
import math

In [ ]:
cont = Contractions(api_key="glove-twitter-25")
cont.load_models()

# Read in Data

In [ ]:
wtnv = pd.read_csv('episode_prelim_clean.csv', sep='|')
texts = wtnv['text']

In [ ]:
f = open("wtnv_ner.txt","r+")  
n = str(f.read())
f.close()

In [ ]:
named_entities = []
bad = ['','Voiced Characters','You (character)','You (listener)','Antiques','Deer','Plastic Bags','Spiders','Them',
      'Shame','LGBT characters','Mayors of Night Vale','Scientists of Night Vale']
bad_start = ['The','Night','Abandoned','Antiques','Barista','Big','City','Coyote','Dark','Desert','Elementary','Haunted',
            'Hidden','Hole','Juvenile','King','Mission','Museum','National','Old','Pine','Post','Pulsar','Radon','Red',
            'Sand','Secret','Shambling','Skeleton','Stone','Used','Air-Filled','Apache','Barks','Cactus','Dr.','Enormous',
            'Glow','Greater','Hierarchy','Hooded','Mr.','Mrs.','My','Otherworldly','Pastor','Prince','Sheriff',
             'Soundproof','Station','Vague,','Whispering','White','World']
lines = n.split('\n')
lines.sort()
for ln in lines:
    nam = ln.split('::')[0]
    if nam not in bad:
        if len(nam.split(' ')) > 1:
            if nam.split(' ')[0] not in bad_start:
                if "'" in nam.split(' ')[0]:
                    named_entities.append(nam.split(' ')[0].split("'")[0])
                elif nam.split(' ')[0] == 'Intern':
                    named_entities.append(nam.split(' ')[1])
                else:
                    named_entities.append(nam.split(' ')[0])
        named_entities.append(nam.split('(')[0].strip().lower())
        
named_entities = list(set([n.lower() for n in named_entities]))

# Functions

In [ ]:
def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

def clear_musics(text):
    musics = re.findall("\[“.*\]", text)
    if len(musics) > 0:
        mus = musics[0].strip().split(']')[0]+']'
        text = text.replace(mus,'')
        musics = re.findall("\[“.*\]", text)
        if len(musics) > 0:
            mus = musics[0].split(']')[0]+']'
            text = text.replace(mus,'')
    
    return text

def clear_note(text):
    if 'NOTE' in text:
        if 'You can purchase' in text:
            text = '1.INTRO.'+text.split('1.INTRO.')[1]
        else:
            text = text.split('page.]')[1]
            
    return text

def clear_links(text):
    if 'http' in text:
        if 'Dinosaur emoji' in text:
            text = text.split('Dinosaur emoji')[0]
        else:
            if 'Mucca' in text:
                text = text.replace("[Weather: “J'Accuse” by Mucca Pazza. http://www.muccapazza.com]",'')
            else:
                if 'Mal' in text:
                    text = text.replace("“See Me” by Mal Blum https://malblum.bandcamp.com/ [I couldn’t find this song on there, but check out the rest of their stuff]",'')
    return text

def clear_bad(text):
    bad = ['for this episode', 'throughout the transcript', 'cecilspeaks', 
           "Transcriber's", 'transcriber','transcribe']
    for sub in bad:
        if sub in text:
            if sub == bad[0]:
                text = 'We'+text.split('] We')[1]
            elif sub == bad[1]:
                text = 'Act'+text.split('] Act')[1]
            elif sub == bad[2]:
                if ('@cecilbaldwin-fan' in text) & ('iTunes' in text):
                    text = text.split('as Earl Harlan.Enjoy. ')[1].split('bye Earl. ')[0]+'bye Earl.'
                else:
                    text = ''
            elif sub == bad[3]:
                text = text.split('headphones] ')[1]
            elif sub == bad[4]:
                text = text.split(' Joseph Fink: Alice')[0].split(' Clip from')[0]
            elif sub == bad[5]:
                if 'following transcript' in text:
                    text = text.split('links] ')[1]
                else:
                    if 'except for the parts that are.  ' in text:
                        text = text.split('except for the parts that are.  ')[1]
                
    return text

def remove_notes(text):
    s = text
    op = s.find('[')
    clo = s.find(']')
    while op != -1:
        s = s.replace(s[op:clo+1],'')
        op = s.find('[')
        clo = s.find(']')
    return s.strip()

In [ ]:
def spellcheck(st,f):
    fixes = f
    
    pos_toks = NER_fix(named_entities,[st.lower()])
    all_toks = []
    for tok in pos_toks[0]:
        all_toks.append(tok[0])

    toks = list(set(all_toks))
    print('%d tokens created...reduced to %d...' %(len(all_toks),len(toks)))
    
    for fix in fixes:
        #print('Applying fix: ',fix)
        target = fix[0]
        while target in all_toks:
            ind = all_toks.index(fix[0])
            all_toks[ind] = fix[1]
    print('Applied preexisting fixes...')
    
    d = {}
    
    for tok in toks:
        d[tok] = suggest(tok)
        
    print('Initial dictionary created...')
    
    d2 = dict(d)
    for word in d2:
        sugs = d2[word]

        if (len(sugs) == 1) and (sugs[0][1] == 1.0) and (sugs[0][0] == word):
            if word in d:
                del d[word]
            
    print('Properly spelled words removed...%d tokens remaining...' %len(d))

    d2 = dict(d)
    for word in d2:
        sugs = d2[word]

        if (word == 'arby') or (word in named_entities) or (word == 'chen'):
            if word in d:
                del d[word]

        elif sugs[0][1] == 1.0:
            fixes.append((word,sugs[0][0]))
            if word in d:
                del d[word]
            
    print('Easy fixes removed...%d tokens remaining...' %len(d))
    
    d2 = dict(d)
    for word in d2:
        first_two = word[:2]
        last_two = word[-2:]
        sugsf1 = suggest(first_two)

        if (len(sugsf1) == 1) and (sugsf1[0][1] == 1.0) and (sugsf1[0][0] == first_two):
            sugsf2 = suggest(word[2:])
            if (len(sugsf2) == 1) and (sugsf2[0][1] == 1.0) and (sugsf2[0][0] == word[2:]):
                fixes.append((word,sugsf1[0][0]+' '+sugsf2[0][0]))
                if word in d:
                    del d[word]

        sugsl1 = suggest(last_two)
        if (len(sugsl1) == 1) and (sugsl1[0][1] == 1.0) and (sugsl1[0][0] == last_two):
            sugsl2 = suggest(word[:-2])
            if (len(sugsl2) == 1) and (sugsl2[0][1] == 1.0) and (sugsl2[0][0] == word[:-2]):
                fixes.append((word,sugsl2[0][0]+' '+sugsl1[0][0]))
                if word in d:
                    del d[word]
                
    print('Two letter mash-ups removed...%d tokens remaining...' %len(d))
    
    d2 = dict(d)
    for word in d2:
        first_three = word[:3]
        last_three = word[-3:]
        sugsf1 = suggest(first_three)

        if (len(sugsf1) == 1) and (sugsf1[0][1] == 1.0) and (sugsf1[0][0] == first_three):
            sugsf2 = suggest(word[3:])
            if (len(sugsf2) == 1) and (sugsf2[0][1] == 1.0) and (sugsf2[0][0] == word[3:]):
                fixes.append((word,sugsf1[0][0]+' '+sugsf2[0][0]))
                if word in d:
                    del d[word]

        sugsl1 = suggest(last_three)
        if (len(sugsl1) == 1) and (sugsl1[0][1] == 1.0) and (sugsl1[0][0] == last_three):
            sugsl2 = suggest(word[:-3])
            if (len(sugsl2) == 1) and (sugsl2[0][1] == 1.0) and (sugsl2[0][0] == word[:-3]):
                fixes.append((word,sugsl2[0][0]+' '+sugsl1[0][0]))
                if word in d:
                    del d[word]
                
    print('Three letter mash-ups removed...%d tokens remaining...' %len(d))
    
    d2 = dict(d)
    for word in d2:
        sugs = d2[word]

        if sugs[0][1] >= 0.75:
            fixes.append((word,sugs[0][0]))
            if word in d:
                del d[word]
            
    print('Okay fixes removed...%d tokens remaining...' %len(d))
    
    d2 = dict(d)
    for word in d2:
        good = False
        word1 = reduce_lengthening(word)
        if word != word1:
            sug = suggest(word1)
            if (sug[0][1] >= 0.75):
                fixes.append((word,sug[0][0]))
                if word in d:
                    del d[word]
                
    print('Elongated fixes removed...%d tokens remaining...' %len(d))
    
    if len(d) <= 100:
        d2 = dict(d)
        for word in d2:
            good = False
            word1 = reduce_lengthening(word)
            for i in range(1,len(word)):
                if good == False:
                    start = word[0:i]
                    end = word[i:]

                    sug_s = suggest(start)
                    if (len(sug_s) == 1) and (sug_s[0][1] == 1.0) and (sug_s[0][0] == start):
                        sug_e = suggest(end)
                        if (len(sug_e) == 1) and (sug_e[0][1] == 1.0) and (sug_e[0][0] == end):
                            fixes.append((word,start+' '+end))
                            if word in d:
                                del d[word]
                            good = True
                        
    print('Remaining mash-ups removed...%d tokens remaining...' %len(d))
    
    for fix in fixes:
        #print('Applying fix: ',fix)
        target = fix[0]
        while target in all_toks:
            ind = all_toks.index(fix[0])
            all_toks[ind] = fix[1]
            
    print('Tokens fixed...')
    
    t = ' '.join(all_toks)
    
    return t,fixes

In [ ]:
def contract_handle(st):
    #print('Doing text %d...' %texts.index(st))
    t = list(cont.expand_texts([st.replace("’","'")]))[0]
    tags = nltk.pos_tag(word_tokenize(str(t)))
    temp = []
    #print('%d tags created...' %len(tags))
    for tag in tags:
        temp.append(tag[0])
            
    return ' '.join(temp)

# Data Cleaning

In [ ]:
texts = [clear_links(clear_note(clear_musics(t.replace('\xa0',' ').replace('…','...').replace('’',"'")))).strip() for t in texts]
texts = [clear_bad(t.replace('“','"').replace('”','"').replace('\t',' ').replace('\n',' ')) for t in texts]

In [ ]:
texts[135] = texts[135]+']'
texts[141] = texts[141]+']'
line = texts[154]
index = line.find('clears throat]')
texts[154] = line[:index] + '[ ' + line[index:]
texts[154] = texts[154]+']'
texts[156] = texts[156]+']'
texts[163] = texts[163]+']'
texts[12] = texts[12].replace('Translations : ( 1 ) You are in danger . ( 2 ) they are coming . ( 3 ) They will come from below . Pies will not help . Russian phrases pieced together using information from the Welcome to Night Vale Wiki , and adding minor variations in Google Translate until the audio preview on the site sounded , to my ear , more or less like what I was hearing in the episode . Then solorwind was kind enough to send me the actual phrases . Any errors are 100 % my fault .','')

In [ ]:
texts = [remove_notes(t) for t in texts]

# SpellCheck

In [ ]:
fixes = [('willbe','will be')]

In [ ]:
for i in range(len(texts)):
    print('Processing Text %d of %d...' %(i+1,len(texts)))
    print('****************************************************************')
    texts[i],fixes = spellcheck(texts[i],fixes)
    print('Length fixes: ',len(fixes))
    print('****************************************************************')
    print()

# Contraction Handler

In [ ]:
texts = [contract_handle(te.replace(" ' ","'")) for te in texts]

# DataFrame (okay)

In [ ]:
len(texts)

In [ ]:
wtnv['text'] = texts

In [ ]:
wtnv

In [ ]:
#wtnv = wtnv.reset_index()
wtnv_2 = pd.DataFrame()
wtnv_2['Episode'] = wtnv['episode_name']
wtnv_2['Text'] = wtnv['text']
wtnv_2['Source'] = 'WTNV'

In [ ]:
wtnv_2.to_csv(r'C:\Users\cml4603\Desktop\twilightvalefalls\wtnv_cleaned.csv', sep='|')